##### 1. Using the OpenCV library, please write a code (Prob1.py) for drawing clock images given two arguments (ie. hour, minute) and meeting the conditions, as follows

In [1]:
"""
Prob1.py - 시계 이미지 생성기
OpenCV를 사용하여 주어진 시(hour)와 분(minute)에 해당하는 시계 이미지를 생성합니다.
랜덤 색상, 노이즈, 크기 조정, 이동 등의 조건을 충족합니다.
"""

import cv2
import numpy as np
import math
import os

# ===== 상수 정의 =====
IMG_SIZE = 227          # 이미지 크기 (227x227x3)
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.5
FONT_THICKNESS = 1
HOUR_HAND_THICKNESS = 5    # 시침 두께
MINUTE_HAND_THICKNESS = 3  # 분침 두께


def generate_random_color(exclude_color=None):
    """
    랜덤 색상 생성 (검정색 제외).
    exclude_color가 주어지면 해당 색상과 충분히 다른 색상을 생성합니다.
    """
    while True:
        color = tuple(np.random.randint(30, 256, 3).tolist())
        if exclude_color is None:
            return color
        # 두 색상 간의 유클리드 거리가 충분히 큰지 확인
        diff = sum((a - b) ** 2 for a, b in zip(color, exclude_color)) ** 0.5
        if diff > 80:
            return color


def add_noise(img):
    """
    이미지에 랜덤 노이즈를 추가합니다.
    노이즈 레벨은 확률에 따라 선택됩니다.
    """
    noise_level = np.random.choice([1, 64, 128, 192, 256], p=[0.1, 0.2, 0.4, 0.2, 0.1])
    noise = np.random.randint(0, noise_level, (IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
    img = cv2.add(img, noise)
    return img


def get_hand_endpoint(cx, cy, length, angle_deg):
    """
    시계 중심(cx, cy)에서 주어진 길이와 각도로 바늘 끝점 좌표를 계산합니다.
    angle_deg: 12시 방향을 0도로 하여 시계 방향으로 증가
    """
    angle_rad = math.radians(angle_deg - 90)
    ex = int(cx + length * math.cos(angle_rad))
    ey = int(cy + length * math.sin(angle_rad))
    return (ex, ey)


def draw_clock_full(hour, minute, clock_size, bg_color, clock_color):
    """
    전체 크기(clock_size x clock_size)의 시계 이미지를 그립니다.
    반환: clock_size x clock_size x 3 크기의 numpy 배열
    """
    clock_img = np.full((clock_size, clock_size, 3), clock_color, dtype=np.uint8)
    cx, cy = clock_size // 2, clock_size // 2
    side = clock_size

    # ===== 1~12 숫자 배치 =====
    radius_text = side * 0.4
    for num in range(1, 13):
        angle_deg = num * 30
        angle_rad = math.radians(angle_deg - 90)
        tx = int(cx + radius_text * math.cos(angle_rad))
        ty = int(cy + radius_text * math.sin(angle_rad))
        text = str(num)
        (tw, th), _ = cv2.getTextSize(text, FONT, FONT_SCALE, FONT_THICKNESS)
        tx -= tw // 2
        ty += th // 2
        cv2.putText(clock_img, text, (tx, ty), FONT, FONT_SCALE, (0, 0, 0),
                     FONT_THICKNESS, cv2.LINE_AA)

    # ===== 시침 그리기 =====
    hour_angle = (hour % 12) * 30 + minute * 0.5
    hour_length = side * 0.25
    hour_end = get_hand_endpoint(cx, cy, hour_length, hour_angle)
    cv2.line(clock_img, (cx, cy), hour_end, (0, 0, 0), HOUR_HAND_THICKNESS)

    # ===== 분침 그리기 =====
    minute_angle = minute * 6
    minute_length = side * 0.425
    minute_end = get_hand_endpoint(cx, cy, minute_length, minute_angle)
    cv2.line(clock_img, (cx, cy), minute_end, (0, 0, 0), MINUTE_HAND_THICKNESS)

    return clock_img


def generate_clock_image(hour, minute, apply_transform=True):
    """
    시계 이미지를 생성합니다.
    Args:
        hour (int): 시 (1~12)
        minute (int): 분 (0~59)
        apply_transform (bool): 크기 조정 및 이동 적용 여부
    Returns:
        img (numpy array): 227x227x3 크기의 시계 이미지
    """
    bg_color = generate_random_color()
    clock_color = generate_random_color(exclude_color=bg_color)

    if apply_transform:
        # 1. 시계 크기: 이미지 크기의 0.3~0.9배
        scale = np.random.uniform(0.3, 0.9)
        clock_size = int(IMG_SIZE * scale)

        # 2. 이동: 시계 변 길이의 0.25~0.5배 범위 내에서 랜덤 이동
        shift_ratio = np.random.uniform(0.25, 0.5)
        shift_amount = int(clock_size * shift_ratio)
        direction = np.random.choice(['up', 'down', 'left', 'right'])

        base_x = (IMG_SIZE - clock_size) // 2
        base_y = (IMG_SIZE - clock_size) // 2

        if direction == 'up':
            base_y -= shift_amount
        elif direction == 'down':
            base_y += shift_amount
        elif direction == 'left':
            base_x -= shift_amount
        elif direction == 'right':
            base_x += shift_amount

        # 3. 시계가 이미지 경계를 벗어나지 않도록 클리핑
        base_x = max(0, min(base_x, IMG_SIZE - clock_size))
        base_y = max(0, min(base_y, IMG_SIZE - clock_size))

        clock_img = draw_clock_full(hour, minute, clock_size, bg_color, clock_color)
        img = np.full((IMG_SIZE, IMG_SIZE, 3), bg_color, dtype=np.uint8)
        img[base_y:base_y + clock_size, base_x:base_x + clock_size] = clock_img
    else:
        clock_img = draw_clock_full(hour, minute, IMG_SIZE, bg_color, clock_color)
        img = clock_img

    img = add_noise(img)
    return img


def generate_dataset(num_samples, save_dir, apply_transform=True):
    """
    학습/테스트용 데이터셋을 생성합니다.
    """
    os.makedirs(save_dir, exist_ok=True)
    labels = []
    for i in range(num_samples):
        hour = np.random.randint(1, 13)
        minute = np.random.randint(0, 60)
        img = generate_clock_image(hour, minute, apply_transform=apply_transform)
        filename = f"{i:04d}_{hour:02d}_{minute:02d}.png"
        cv2.imwrite(os.path.join(save_dir, filename), img)
        labels.append((hour, minute))
    return labels


if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description="시계 이미지 생성기")
    parser.add_argument("--hour", type=int, default=3, help="시 (1~12)")
    parser.add_argument("--minute", type=int, default=0, help="분 (0~59)")
    parser.add_argument("--num_samples", type=int, default=1, help="생성할 샘플 수")
    parser.add_argument("--save_dir", type=str, default="./generated_clocks", help="저장 디렉토리")
    parser.add_argument("--no_transform", action="store_true", help="크기 조정/이동 비활성화")
    args = parser.parse_args()

    if args.num_samples == 1:
        img = generate_clock_image(args.hour, args.minute, apply_transform=not args.no_transform)
        os.makedirs(args.save_dir, exist_ok=True)
        save_path = os.path.join(args.save_dir, f"clock_{args.hour:02d}_{args.minute:02d}.png")
        cv2.imwrite(save_path, img)
        print(f"시계 이미지 저장됨: {save_path}")
    else:
        labels = generate_dataset(args.num_samples, args.save_dir,
                                   apply_transform=not args.no_transform)
        print(f"{args.num_samples}개의 시계 이미지가 {args.save_dir}에 저장되었습니다.")

usage: ipykernel_launcher.py [-h] [--hour HOUR] [--minute MINUTE]
                             [--num_samples NUM_SAMPLES] [--save_dir SAVE_DIR]
                             [--no_transform]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-c9bd2776-0221-4986-b17b-3593157a2a0e.json


SystemExit: 2

/root/anaconda3/envs/cv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
"""
Prob2.py - 시계 시간 예측 딥러닝 네트워크 정의
ResNet18을 기반으로 시침/분침의 각도를 sin/cos 값으로 예측하는 회귀 네트워크입니다.

접근 방식:
- 시침 각도와 분침 각도를 각각 sin, cos 값으로 변환하여 예측 (총 4개 출력)
- 각도를 직접 예측하면 0도와 360도 경계에서 불연속성이 발생하므로,
  sin/cos 표현을 사용하여 이 문제를 해결합니다.
- 사전학습된 ResNet18을 사용하여 전이학습 수행
"""

import torch
import torch.nn as nn
import torchvision.models as models
import math


class ClockNet(nn.Module):
    """
    시계 시간 예측 네트워크
    
    입력: 227x227x3 시계 이미지
    출력: [hour_sin, hour_cos, minute_sin, minute_cos] (4개 값)
    
    시침 각도: (hour % 12) * 30 + minute * 0.5  → 0~360도
    분침 각도: minute * 6  → 0~360도
    """
    
    def __init__(self, pretrained=True):
        super(ClockNet, self).__init__()
        
        # ===== 사전학습된 ResNet18 백본 =====
        self.backbone = models.resnet18(pretrained=pretrained)
        
        # ResNet18의 마지막 FC 레이어 입력 차원 (512)
        num_features = self.backbone.fc.in_features
        
        # 기존 FC 레이어를 새로운 회귀 헤드로 교체
        # sin/cos 표현으로 4개 출력 (hour_sin, hour_cos, minute_sin, minute_cos)
        self.backbone.fc = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 4),    # [hour_sin, hour_cos, minute_sin, minute_cos]
            nn.Tanh()            # sin/cos 값은 -1~1 범위이므로 Tanh 사용
        )
    
    def forward(self, x):
        """
        순전파
        Args:
            x: [B, 3, 227, 227] 크기의 입력 텐서
        Returns:
            output: [B, 4] 크기의 출력 텐서
        """
        return self.backbone(x)


# ===== 유틸리티 함수들 =====

def time_to_angles(hour, minute):
    """시간을 각도(도)로 변환합니다."""
    hour_angle = (hour % 12) * 30.0 + minute * 0.5
    minute_angle = minute * 6.0
    return hour_angle, minute_angle


def angles_to_sincos(hour_angle, minute_angle):
    """각도를 sin/cos 값으로 변환합니다."""
    h_rad = math.radians(hour_angle)
    m_rad = math.radians(minute_angle)
    return torch.tensor([
        math.sin(h_rad), math.cos(h_rad),
        math.sin(m_rad), math.cos(m_rad)
    ], dtype=torch.float32)


def sincos_to_angles(output):
    """네트워크 출력(sin/cos)을 각도로 변환합니다."""
    hour_sin, hour_cos, minute_sin, minute_cos = output
    hour_angle = math.degrees(math.atan2(hour_sin, hour_cos)) % 360
    minute_angle = math.degrees(math.atan2(minute_sin, minute_cos)) % 360
    return hour_angle, minute_angle


def angles_to_time(hour_angle, minute_angle):
    """각도를 시간으로 변환합니다."""
    minute = round(minute_angle / 6.0) % 60
    hour = round((hour_angle - minute * 0.5) / 30.0) % 12
    if hour == 0:
        hour = 12
    return hour, minute


def predict_time(output):
    """네트워크 출력에서 시간을 예측합니다."""
    if isinstance(output, torch.Tensor):
        output = output.detach().cpu().numpy()
    hour_angle, minute_angle = sincos_to_angles(output)
    hour, minute = angles_to_time(hour_angle, minute_angle)
    return hour, minute


class ClockLoss(nn.Module):
    """시계 시간 예측용 손실 함수 (MSE Loss)"""
    def __init__(self):
        super(ClockLoss, self).__init__()
        self.mse = nn.MSELoss()
    
    def forward(self, pred, target):
        return self.mse(pred, target)


if __name__ == "__main__":
    # 변환 함수 테스트
    for h in [1, 3, 6, 9, 12]:
        for m in [0, 15, 30, 45, 59]:
            h_a, m_a = time_to_angles(h, m)
            sc = angles_to_sincos(h_a, m_a)
            ph, pm = predict_time(sc.numpy())
            ok = "OK" if (ph == h and pm == m) else "FAIL"
            print(f"{ok} {h:2d}:{m:02d} -> {ph:2d}:{pm:02d}")

ModuleNotFoundError: No module named 'torch'